In [1]:
import os
os.chdir('../')

In [2]:
!pwd

/Users/shrey/Desktop/github/butterfly_classifier


In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data_dir: Path
    training_data_csv: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_classes: int
    params_learning_rate: float


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_dir
import tensorflow as tf
tf.compat.v1.enable_eager_execution()


class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_dir([self.config.artifacts_root])

    

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_dir([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
    
        return prepare_callback_config

    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        #training_data = os.path.join(self.config.data_ingestion.unzip_dir, 'data/train')
        create_dir([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data_dir=Path(training.training_data_dir),
            training_data_csv=Path(training.training_data_csv),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_classes=params.CLASSES,
            params_learning_rate=params.LEARNING_RATE
        )
        return training_config


In [5]:
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
            timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
            tb_running_log_dir = os.path.join(
                self.config.tensorboard_root_log_dir,
                f'tb_logs_at_{timestamp}'
            )
            return tf.keras.callbacks.TensorBoard(
                log_dir=tb_running_log_dir,
                histogram_freq=1
            )

    @property
    def _create_checkpoint_callbacks(self):
            return tf.keras.callbacks.ModelCheckpoint(
                filepath=self.config.checkpoint_model_filepath,
                save_best_only=True
            )
        
    def get_tb_ckpt_callbacks(self):
            return [
                self._create_tb_callbacks, 
                self._create_checkpoint_callbacks
                ]

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import time
import pandas as pd
import numpy as np


class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        # ✅ Force-enable eager execution at the beginning
        tf.config.run_functions_eagerly(True)
        print(f"✅ Eager Execution Enabled: {tf.executing_eagerly()}\n")


    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
        # ✅ Debugging Step: Print Model Output
        print(f"✅ Model Loaded: Output shape {self.model.output_shape}")

        # ✅ Ensure output layer matches number of classes
        expected_classes = self.config.params_classes

        if self.model.output_shape[-1] != expected_classes:
            print(f"🔄 Rebuilding Model: Expected {expected_classes} classes, found {self.model.output_shape[-1]}")
            
            # ✅ Remove the last layer
            base_model = tf.keras.models.Model(inputs=self.model.input, outputs=self.model.layers[-2].output)
            
            # ✅ Add new classification head
            new_output = tf.keras.layers.Dense(expected_classes, activation="softmax")(base_model.output)
            self.model = tf.keras.models.Model(inputs=base_model.input, outputs=new_output)

            print(f"✅ Model rebuilt with {expected_classes} output classes")

        # ✅ Compile the Model Again
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

        print("✅ Model Recompiled Successfully")



    # def get_base_model(self):
    #     self.model = tf.keras.models.load_model(
    #         self.config.updated_base_model_path
    #     )


    def train_valid_generator(self):

        # ✅ Debugging: Print expected file path
        abs_path = os.path.abspath(self.config.training_data_csv)
        print(f"🔍 Checking file existence: {abs_path}")

        if not os.path.exists(self.config.training_data_csv):
            raise FileNotFoundError(f"❌ CSV file not found at {self.config.training_data_csv}\n"
                                    f"🔹 Expected Absolute Path: {abs_path}")

        print(f"✅ CSV file found: {abs_path}")
        train_df = pd.read_csv(self.config.training_data_csv)

        train_df['filename'] = train_df['filename'].apply( 
            lambda x: os.path.join(self.config.training_data_dir, x)
        )

        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.2
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='categorical',
            interpolation='bilinear'
        ) 

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs)
        
        self.valid_generator = valid_datagenerator.flow_from_dataframe(
            dataframe=train_df,
            x_col='filename',
            y_col='label',
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

        # self.valid_generator = valid_datagenerator.flow_from_directory(
        #     directory=self.config.training_data,
        #     subset='validation',
        #     shuffle=False,
        #     **dataflow_kwargs
        # )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_dataframe(
            dataframe=train_df,
            x_col='filename',
            y_col='label',
            subset='training',
            shuffle=True,
            **dataflow_kwargs
        )
        print(f'Found {self.train_generator.samples} training images')
        print(f'Found {self.valid_generator.samples} validation images')
        
        # self.train_generator = train_datagenerator.flow_from_directory(
        #     directory=self.config.training_data,
        #     subset='training',
        #     shuffle=True,
        #     **dataflow_kwargs
        # )

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        # ✅ Ensure optimizer is recompiled
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )
        
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        print(f"🚀 Training Started for {self.config.params_epochs} epochs with {self.steps_per_epoch} steps per epoch")

        # ✅ Train the model
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        print("✅ Training Complete!")
        self.save_model(path=self.config.trained_model_path, model=self.model)



    # def train(self,callback_list: list):
    #     self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
    #     self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

    #     self.model.fit(
    #         self.train_generator,
    #         epochs=self.config.params_epochs,
    #         steps_per_epoch=self.steps_per_epoch,
    #         validation_steps=self.validation_steps,
    #         validation_data=self.valid_generator,
    #         callbacks=callback_list
    #     )

    #     self.save_model(path=self.config.training_model_path, model=self.model)


In [7]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callback_list)

except Exception as e:
    raise e

[2025-01-29 16:14:15,270: INFO: common: YAML file loaded successfully: config/config.yaml]
[2025-01-29 16:14:15,271: INFO: common: YAML file loaded successfully: params.yaml]
[2025-01-29 16:14:15,272: INFO: common: Directory created successfully: artifacts]
[2025-01-29 16:14:15,272: INFO: common: Directory created successfully: artifacts/prepare_callbacks/checkpoint_dir]
[2025-01-29 16:14:15,273: INFO: common: Directory created successfully: artifacts/prepare_callbacks/tensorboard_log_dir]
[2025-01-29 16:14:15,273: INFO: common: Directory created successfully: artifacts/training]
✅ Eager Execution Enabled: True

[2025-01-29 16:14:15,395: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
✅ Model Loaded: Output shape (None, 75)
✅ Model Recompiled Successfully
🔍 Checking file existence: /Users/shrey/Desktop/github/butterfly_classifier/artifacts/data_ingestion/data/T

/opt/anaconda3/envs/flyenv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/opt/anaconda3/envs/flyenv/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


325/325 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.2336 - loss: 4.6367

/opt/anaconda3/envs/flyenv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/opt/anaconda3/envs/flyenv/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


[2025-01-29 16:43:26,242: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
325/325 ━━━━━━━━━━━━━━━━━━━━ 1751s 5s/step - accuracy: 0.2340 - loss: 4.6332 - val_accuracy: 0.5401 - val_loss: 2.0968
✅ Training Complete!
[2025-01-29 16:43:26,297: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
